In [116]:
from SLGCA_Model import SLGCA
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import numpy as np
import warnings
from preprecess import *

In [117]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
os.environ['R_HOME'] = '/home/luxin1/miniconda3/envs/pytorch/lib/R'

sample_name = 151673
file_fold = '/home/luxin1/DLPFC/' + str(sample_name)

n_clusters = 7

In [118]:
adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)
adata.var_names_make_unique()
df_meta = pd.read_csv(file_fold + "/metadata.tsv", sep="\t")
de_meta_layer = df_meta['layer_guess']
adata.obs['ground_truth'] = de_meta_layer.values
adata = adata[~pd.isnull(adata.obs['ground_truth'])]

In [119]:
model = SLGCA(adata = adata, device=device, epochs=500,dim_output=64, n_top_genes=4000, n_neighbors=6,alpha=10,beta=0.5,gama=0.5)
adata = model.train()

Begin to train ST data...


100%|██████████| 500/500 [00:12<00:00, 39.86it/s]

Optimization finished for ST data!


In [120]:
from utils import clustering
tool = 'mclust'
radius = 50
clustering(adata, n_clusters=n_clusters, method=tool, radius=radius, refinement=True)

ARI = metrics.adjusted_rand_score(adata.obs['SLGCA'], adata.obs['ground_truth'])
NMI = metrics.normalized_mutual_info_score(adata.obs['SLGCA'], adata.obs['ground_truth'])
print(ARI, NMI)

fitting ...
  |======================================================================| 100%
0.6588763409300818 0.7474066191150414


In [121]:
adata

AnnData object with n_obs × n_vars = 3611 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'ground_truth', 'mclust', 'SLGCA'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'spatial', 'hvg', 'log1p'
    obsm: 'spatial', 'feat', 'distance_matrix', 'graph_neigh', 'emb', 'rec', 'emb_pca'

In [122]:
adata.write(file_fold + "/adata.h5ad")